# Horologium

### Creating a digital clock for Roman-style date-and-timekeeping, anywhere in the world. 
-------------------------------
Jacob Schwartz

Professor Pletcher

Intro to Digital Humanities

May 2, 2025


For this assignment, my objective was to create a digital clock for the Romans, who had a way of reckoning time based solely on the movement of the sun. In addition, this clock displays the current date based on how the Romans expressed it, in relation to the Kalends, the Nones, or the Ides of a given month. Creating this involved the use of Python's datetime module, multiple APIs, and command-line prompts. 

## Roman Timekeeping
Let's start with how the Romans told time over the course of a day. One of our best sources for this is Censorinus in *De Die Natali (Liber),* written around 238 CE.<sup>1</sup> According to him, *in horas XII diem divisum esse noctemque in totidem vulgo notum est* ("that the day and night are [each] divided into twelve hours is known among all the people").<sup>2</sup> The "day" (*naturaliter dies*) is reckoned from sunrise to sunset, while the "night" (*nox*) is sunset to sunrise.<sup>3</sup> As the day lengthens and shortens with the changing of the seasons, so too do the hours: we have twelve short hours of daylight in the winter and twelve long hours in the summer. In addition, the length of these hours varies based on latitude.<sup>4</sup> A soldier on Hadrian's wall in wintertime, for instance, would experience far shorter hours than his compatriot in Alexandria on the same day.

---------

1: Censorinus, *De Die Natali,* trans. William Maude. 

2: Censorinus, *De Die Natali Liber* 23.6. 

3: Censorinus 23.2. 

4: Schmitz, Leonhard, "Hora." 

## Calculating the Day
To calculate 12 equal hours from sunrise to sunset, then, we will need two thing. The first is SunriseSunset.io, an API which finds the time of sunrise and sunset for a given latitude and longitude. The second thing we will need is the datetime module, which will allow us to get the current date and time as well as calculate the difference between times (called a timedelta). 

In [ ]:
# First, we'll import our modules.
import datetime
from datetime import timezone
import requests

# Next, we're going to define our time now. Later, we'll use the module to get the current time, but for now, let's say it's 
# 12:00 PM on March 15th (the Ides!). 
timenow = datetime.datetime(2025, 3, 15, 12, 0, 0, 000000)


# Then we'll isolate the date and convert it into a string. We're going to need to use this a lot. 
thedate = timenow.date().strftime('%Y-%m-%d')

# These look like this:
print(timenow)
print(thedate)

In [ ]:
# Now let's use our API to find the sunrise and sunset at a given location. For now, let's pick a spot in London.
# Let's use the City of London, the neighborhood built on the remains of the ancient city of Londinium.
lat = '51.515556'
lng = '-0.093056'

# Powered by SunriseSunset.io: https://sunrisesunset.io/api/
def make_sun_request(lat, lng):
    url = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={thedate}&time_format=24"
    r = requests.get(url)
    response = r.json()
    sunrise = response['results']['sunrise']
    sunset = response['results']['sunset']

    return [sunrise, sunset]

make_sun_request(lat, lng)

In [ ]:
# Since these are strings, we need to convert them to datetimes to do calculations with them. 
def convert_sun_to_datetime(sunrise, sunset):
    risetime = datetime.datetime.strptime(f"{thedate}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
    settime = datetime.datetime.strptime(f"{thedate}, {sunset}", "%Y-%m-%d, %H:%M:%S")
    return [risetime, settime]

def get_datetimes_from_coord(lat, lng):
    srss = make_sun_request(lat, lng)
    sunrise = srss[0]
    sunset = srss[1]
    datetimes = convert_sun_to_datetime(sunrise, sunset)
    return datetimes

datetimes = get_datetimes_from_coord(lat, lng)
datetimes

In [ ]:
# Now we can calculate our twelve hours of the day. 
def calc_hodie(risetime, settime):
    dies = settime-risetime
    hora = dies/12
    hora_prima = risetime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_secunda:
        tempus = "\nI\nprima diei hora"
    elif hora_secunda <= timenow < hora_tertia:
        tempus = "\nII\nseconda diei hora"
    elif hora_tertia <= timenow < hora_quarta:
        tempus = "\nIII\ntertia diei hora"
    elif hora_quarta <= timenow < hora_quinta:
        tempus = "\nIV\nquarta diei hora"
    elif hora_quinta <= timenow < hora_sexta:
        tempus = "\nV\nquinta diei hora"
    elif hora_sexta <= timenow < hora_septima:
        tempus = "\nVI\nsexta diei hora"
    elif hora_septima <= timenow < hora_octava:
        tempus = "\nVII\nseptima diei hora"
    elif hora_octava <= timenow < hora_nona:
        tempus = "\nVIII\noctava diei hora"
    elif hora_nona <= timenow < hora_decima:
        tempus = "\nIX\nnona diei hora"
    elif hora_decima <= timenow < hora_undecima:
        tempus = "\nX\ndecima diei hora"
    elif hora_undecima <= timenow < hora_duodecima:
        tempus = "\nXI\nundecima diei hora"
    elif hora_duodecima <= timenow < settime:
        tempus = "\nXII\nduodecima diei hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the day!"
    return tempus

risetime = datetimes[0]
settime = datetimes[1]
tempus = calc_hodie(risetime, settime)

print(tempus)

## Calculating the Night
Before we get to calculating the time between sunset and sunrise, we need to talk about some other ways of reckoning time among the Romans. In particular, Censorinus mentions that *alii diem quadripertito, sed et noctem similiter dividebant; idque similitudo testatur militaris, ubi dicitur vigilia prima, item secunda et tertia et quarta* ("others, with the day having been divided in four, even divided the night similarly; and this is witnessed by the similar [divisions] in military language, where "first watch," is said, then "second" and "third" and "fourth").<sup>5</sup> The four watches, particularly those of the night, are an especially significant alternative, appearing frequently in the New Testament as well.<sup>6</sup> For this reason, we're going to include them too, with each watch being three hours long.<sup>7</sup>

With this in mind, let's write some code that will divide the night into hours. 

-----------

5: Censorinus 23.9.

6: Matthew 14:25, Luke 12:38, Mark 13:35.

7: Adam, *Roman Antiquities,* 307.

In [ ]:
# Now that we're covering the entire 24-hour day, we can get the actual current time. We're using UTC, for reasons we will
# return to later.
timenowtz = datetime.datetime.now(timezone.utc)
timenow = timenowtz.replace(tzinfo=None)

thedate = timenow.date().strftime('%Y-%m-%d')

# In order to divide the night into twelve, we need not only the sunrise and sunset time of our current day but also sunset
# the evening before and sunrise the morning after. Therefore, we need to make a request for the day before and after as well.
daybefore = timenow - datetime.timedelta(days=1)
datebefore = daybefore.date().strftime('%Y-%m-%d')
dayafter = timenow + datetime.timedelta(days=1)
dateafter = dayafter.date().strftime('%Y-%m-%d')

# Powered by SunriseSunset.io: https://sunrisesunset.io/api/
def make_sun_request(lat, lng):
    url = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={thedate}&time_format=24"
    r = requests.get(url)
    response = r.json()
    sunrise = response['results']['sunrise']
    sunset = response['results']['sunset']

    url2 = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={datebefore}&time_format=24"
    r2 = requests.get(url2)
    response2 = r2.json()
    nightstart = response2['results']['sunset']

    url3 = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={dateafter}&time_format=24"
    r3 = requests.get(url3)
    response3 = r3.json()
    nightend = response3['results']['sunrise']

    return [nightstart, sunrise, sunset, nightend]

def convert_sun_to_datetime(nightstart, sunrise, sunset, nightend):
    starttime = datetime.datetime.strptime(f"{datebefore}, {nightstart}", "%Y-%m-%d, %H:%M:%S")
    risetime = datetime.datetime.strptime(f"{thedate}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
    settime = datetime.datetime.strptime(f"{thedate}, {sunset}", "%Y-%m-%d, %H:%M:%S")
    endtime = datetime.datetime.strptime(f"{dateafter}, {nightend}", "%Y-%m-%d, %H:%M:%S")
    return [starttime, risetime, settime, endtime]

def get_datetimes_from_coord(lat, lng):
    srss = make_sun_request(lat, lng)
    nightstart = srss[0]
    sunrise = srss[1]
    sunset = srss[2]
    nightend = srss[3]
    datetimes = convert_sun_to_datetime(nightstart, sunrise, sunset, nightend)
    return datetimes

def calc_hodie(risetime, settime):
    dies = settime-risetime
    hora = dies/12
    hora_prima = risetime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_secunda:
        tempus = "\nI\nprima diei hora"
    elif hora_secunda <= timenow < hora_tertia:
        tempus = "\nII\nseconda diei hora"
    elif hora_tertia <= timenow < hora_quarta:
        tempus = "\nIII\ntertia diei hora"
    elif hora_quarta <= timenow < hora_quinta:
        tempus = "\nIV\nquarta diei hora"
    elif hora_quinta <= timenow < hora_sexta:
        tempus = "\nV\nquinta diei hora"
    elif hora_sexta <= timenow < hora_septima:
        tempus = "\nVI\nsexta diei hora"
    elif hora_septima <= timenow < hora_octava:
        tempus = "\nVII\nseptima diei hora"
    elif hora_octava <= timenow < hora_nona:
        tempus = "\nVIII\noctava diei hora"
    elif hora_nona <= timenow < hora_decima:
        tempus = "\nIX\nnona diei hora"
    elif hora_decima <= timenow < hora_undecima:
        tempus = "\nX\ndecima diei hora"
    elif hora_undecima <= timenow < hora_duodecima:
        tempus = "\nXI\nundecima diei hora"
    elif hora_duodecima <= timenow < settime:
        tempus = "\nXII\nduodecima diei hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the day!"
    return tempus

# Here we'll calculate the night before. 
# quid proxima, quid superiore nocte egeris (Cic. Cat. 1.1.1)
def calc_nox_proxima(startime, risetime):
    nox = risetime - startime
    hora = nox/12
    hora_prima = startime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_quarta:
        watch = "VIGILIA PRIMA\n"
        if hora_prima <= timenow < hora_secunda:
            tempus = watch + "I\nprima noctis hora"
        elif hora_secunda <= timenow < hora_tertia:
            tempus = watch + "II\nseconda noctis hora"
        elif hora_tertia <= timenow < hora_quarta:
            tempus = watch + "III\ntertia noctis hora"
    elif hora_quarta <= timenow < hora_septima:
        watch = "VIGILIA SECUNDA\n"
        if hora_quarta <= timenow < hora_quinta:
            tempus = watch + "IV\nquarta noctis hora"
        elif hora_quinta <= timenow < hora_sexta:
            tempus = watch + "V\nquinta noctis hora"
        elif hora_sexta <= timenow < hora_septima:
            tempus = watch + "VI\nsexta noctis hora"
    elif hora_septima <= timenow < hora_decima:
        watch = "VIGILIA TERTIA\n"
        if hora_septima <= timenow < hora_octava:
            tempus = watch + "VII\nseptima noctis hora"
        elif hora_octava <= timenow < hora_nona:
            tempus = watch + "VII\noctava noctis hora"
        elif hora_nona <= timenow < hora_decima:
            tempus = watch + "IX\nnona noctis hora"
    elif hora_decima <= timenow < risetime:
        watch = "VIGILIA QUARTA\n"
        if hora_decima <= timenow < hora_undecima:
            tempus = watch + "X\ndecima noctis hora"
        elif hora_undecima <= timenow < hora_duodecima:
            tempus = watch + "XI\nundecima noctis hora"
        elif hora_duodecima <= timenow < risetime:
            tempus = watch + "XII\nduodecima noctis hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the last night!"
    return tempus

# Now we'll calculate the night after
def calc_nox_superior(settime, endtime):
    nox = endtime - settime
    hora = nox/12
    hora_prima = settime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_quarta:
        watch = "VIGILIA PRIMA\n"
        if hora_prima <= timenow < hora_secunda:
            tempus = watch + "I\nprima noctis hora"
        elif hora_secunda <= timenow < hora_tertia:
            tempus = watch + "II\nseconda noctis hora"
        elif hora_tertia <= timenow < hora_quarta:
            tempus = watch + "III\ntertia noctis hora"
    elif hora_quarta <= timenow < hora_septima:
        watch = "VIGILIA SECUNDA\n"
        if hora_quarta <= timenow < hora_quinta:
            tempus = watch + "IV\nquarta noctis hora"
        elif hora_quinta <= timenow < hora_sexta:
            tempus = watch + "V\nquinta noctis hora"
        elif hora_sexta <= timenow < hora_septima:
            tempus = watch + "VI\nsexta noctis hora"
    elif hora_septima <= timenow < hora_decima:
        watch = "VIGILIA TERTIA\n"
        if hora_septima <= timenow < hora_octava:
            tempus = watch + "VII\nseptima noctis hora"
        elif hora_octava <= timenow < hora_nona:
            tempus = watch + "VII\noctava noctis hora"
        elif hora_nona <= timenow < hora_decima:
            tempus = watch + "IX\nnona noctis hora"
    elif hora_decima <= timenow < settime:
        watch = "VIGILIA QUARTA\n"
        if hora_decima <= timenow < hora_undecima:
            tempus = watch + "X\ndecima noctis hora"
        elif hora_undecima <= timenow < hora_duodecima:
            tempus = watch + "XI\nundecima noctis hora"
        elif hora_duodecima <= timenow < settime:
            tempus = watch + "XII\nduodecima noctis hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the next night!"
    return tempus 

def get_tempus_from_coord(lat, lng):
    datetimes = get_datetimes_from_coord(lat, lng)
    starttime = datetimes[0]
    risetime = datetimes[1]
    settime = datetimes[2]
    endtime = datetimes[3]
    if starttime <= timenow < risetime:
        nox_proxima = calc_nox_proxima(starttime, risetime)
        return nox_proxima
    elif risetime <= timenow < settime:
        dies = calc_hodie(risetime, settime)
        return dies
    elif settime <= timenow < endtime:
        nox_superior = calc_nox_superior(settime, endtime)
        return nox_superior
    else:
        return "Eheu! Couldn't get the time from the sun!"
    
# We'll use the same coordinates as before, London, for ease of use with UTC. 
lat = '51.515556'
lng = '-0.093056'
tempus = get_tempus_from_coord(lat, lng)
print(tempus)

## Getting the Date: The Roman Calendar
We now have code which successfully gives us the time (sort of - we'll come back to this in a bit). Now we can move on to printing the date as the Romans would have reckoned it. 

Firstly, to prepare our date, we need to modify it. According to Censorinus, the Romans reckoned their days *a media nocte ad mediam noctem* ("from midnight to midnight").<sup>8</sup> However, this midnight means **solar** midnight, that is, between the sixth and seventh hour of the night. So we need to adjust our date, like so: 

----------------

8: Censorinus 23.3.

In [ ]:
def modify_date(thedate, starttime, risetime, settime, endtime):
    if starttime <= timenow < risetime:
        nox_proxima = risetime - starttime
        dimidium = nox_proxima/2
        media_nox = starttime + dimidium
        if starttime <= timenow < media_nox:
            mod_date = datebefore
        else:
            mod_date = thedate
    elif settime <= timenow < endtime:
        nox_superior = endtime - settime
        dimidium = nox_superior/2
        media_nox = settime + dimidium
        if media_nox <= timenow < endtime:
            mod_date = dateafter
        else:
            mod_date = thedate
    elif risetime <= timenow < settime:
        mod_date = thedate
    else:
        mod_date = "Eheu! Couldn't figure out if it's past midnight!"
    return mod_date

# After modifying the date, we'll split the string like so:
def split_date(mod_date):
    splitdate = mod_date.split("-")
    yearstr = splitdate[0]
    year = int(yearstr)
    monthstr = splitdate[1]
    month = int(monthstr)
    daystr = splitdate[2]
    day = int(daystr)
    return [year, month, day]

datetimes = get_datetimes_from_coord(lat, lng)
starttime = datetimes[0]
risetime = datetimes[1]
settime = datetimes[2]
endtime = datetimes[3]

mod_date = modify_date(thedate, starttime, risetime, settime, endtime)
ymd = split_date(mod_date)
year = ymd[0]
month = ymd[1]
day = ymd[2]
print(year, month, day)

Now we're ready to find our date. To do this, we need to understand the Roman calendar. The Romans stated the day by counting up, inclusively, from the current day to the next of three special days in a month: the kalends, the nones, and the ides. Although the kalends were always the first of the month, which day of the month the other two fell on depended on the month. Originally, in Rome's mythic ten-month calendar predating Numa, four months were considered *pleni,* or "full" (long - these being March, May, Quintilis/August, and October), while the other six were considered *cavi,* or "hollow" (short).<sup>9</sup> These full months had their nones and their ides on the 7th and 15th, respectively, while the hollow months had them on the 5th and 13th. Numa's reform added the hollow months of January and February, while the Julian reforms added days to many of the months, such that January, August, and December now shared the same number of days as the full months, leading to seven months with 31 days - but in spite of this, these reformed months kept their nones and ides on the 5th and 13th.<sup>10</sup>

From this information, we can create the following dictionary of Roman months:

-------------

9: Censorinus 20.3.

10: Censorinus 20. 

In [11]:
menses = {
    'Ianuarius': {
        'name': 'Ianuarius',
        'abbr.':'Ian.',
        'nom.fem.pl.': 'Ianuariae',
        'acc.fem.pl.': 'Ianuarias',
        'abl.pl.': 'Ianuariis',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Februarius': {
        'name': 'Februarius', 
        'abbr.':'Feb.',
        'nom.fem.pl.': 'Februariae',
        'acc.fem.pl.': 'Februarias',
        'abl.pl.': 'Februariis',
        'days': 28, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Februarius (leap year)': {
        'name': 'Februarius', 
        'abbr.':'Feb.',
        'nom.fem.pl.': 'Februariae',
        'acc.fem.pl.': 'Februarias',
        'abl. pl.': 'Februariis',
        'days': 29, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Martius': {
        'name': 'Martius', 
        'abbr.':'Mar.',
        'nom.fem.pl.': 'Martiae',
        'acc.fem.pl.': 'Martias',
        'abl.pl.': 'Martiis',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 7, 
        'idus': 15
        },
    'Aprilis': {
        'name': 'Aprilis', 
        'abbr.':'Apr.',
        'nom.fem.pl.': 'Apriles',
        'acc.fem.pl.': 'Apriles',
        'abl.pl.': 'Aprilibus',
        'days': 30, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Maius': {
        'name': 'Maius', 
        'abbr.':'Mai.',
        'nom.fem.pl.': 'Maiae',
        'acc.fem.pl.': 'Maias',
        'abl.pl.': 'Maiis',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 7, 
        'idus': 15
        },
    'Iunius': {
        'name': 'Iunius',
        'abbr.':'Iun.',
        'nom.fem.pl.': 'Iuniae',
        'acc.fem.pl.': 'Iunias', 
        'abl.pl.': 'Iuniis',
        'days': 30, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Iulius': {
        'name': 'Iulius', 
        'abbr.':'Iul.',
        'nom.fem.pl.': 'Iuliae',
        'acc.fem.pl.': 'Iulias',
        'abl.pl.': 'Iuliis',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 7, 
        'idus': 15
        },
    'Augustus': {
        'name': 'Augustus',
        'abbr.':'Aug.',
        'nom.fem.pl.': 'Augustae',
        'acc.fem.pl.': 'Augustas',
        'abl.pl.': 'Augustis', 
        'days': 31, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'September': {
        'name': 'September',
        'abbr.':'Sept.',
        'nom.fem.pl.': 'Septembres',
        'acc.fem.pl.': 'Septembres',
        'abl.pl.': 'Septembribus', 
        'days': 30, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'October': {
        'name': 'October',
        'abbr.':'Oct.',
        'nom.fem.pl.': 'Octobres',
        'acc.fem.pl.': 'Octobres',
        'abl.pl.': 'Octobribus', 
        'days': 31, 
        'kalendae': 1, 
        'nonae': 7, 
        'idus': 15
        },
    'November': {
        'name': 'November',
        'abbr.':'Nov.',
        'nom.fem.pl.': 'Novembres',
        'acc.fem.pl.': 'Novembres', 
        'abl.pl.': 'Novembribus',
        'days': 30, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'December': {
        'name': 'December', 
        'abbr.':'Dec.',
        'nom.fem.pl.': 'Decembres',
        'acc.fem.pl.': 'Decembres',
        'abl.pl.': 'Decembribus',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13}
        }

A few more details are important for deriving our Roman date. Firstly, Roman inclusive counting meant that they counted the day they were **on**, as well as the day they were counting towards. So in March, they would call March 13th "the third day before the Ides," and March 12th "the fourth day before the Ides," while March 14th was simply "the day before the Ides." We will need to factor this in to our math. 

Secondly, we need to note leap years. Under the Julian reforms, which is what this code should reflect, a leap year occurred every four years. This additional day was inserted not at the end of the month, as in our reckoning, but *post Terminalia... quod nunc bis sextum vocatur* ("after the Terminalia... which is now called 'twice-sixth' [day before the Kalends of March]).<sup>11</sup> In other words, February 24th, the sixth day before the Kalends of March, was counted twice.<sup>12</sup> Our code will have to reflect this too. Even though the Julian calendar had a leap year every four years, however, we don't want our code to be out of sync with our Gregorian calendar, so we will need to utilize the Gregorian calendar's rules for when leap years occur, applying this "bissextile" day in years that are divisible by 400 or which are divisible by 4 and not by 100. 

With all this in mind, we can now write our code. 

------------

11: Censorinus 20.10.

12: "The Roman Calendar."

In [ ]:
# A method for converting integers into Roman numerals. 
# Solution found here: https://stackoverflow.com/questions/28777219/basic-program-to-convert-integer-to-roman-numerals
num_map = [(1000, 'M'), (900, 'CM'), (500, 'D'), (400, 'CD'), (100, 'C'), (90, 'XC'),
           (50, 'L'), (40, 'XL'), (10, 'X'), (9, 'IX'), (5, 'V'), (4, 'IV'), (1, 'I')]

def num2roman(num):
    roman = ''
    while num > 0:
        for i, r in num_map:
            while num >= i:
                roman += r
                num -= i

    return roman


# Latin ordinal numbers, useful for plugging into our printed statements. 
ordinals = {1:'primum', 2:'secundum', 3:'tertium', 4:'quartum', 5:'quintum', 6:'sextum', 
            7:'septimum', 8:'octavum', 9:'nonum', 10:'decimum', 11:'undecimum', 12:'duodecimum', 
            13:'tertium decimum', 14:'quartum decimum', 15:'quintum decimum', 16:'sextum decimum', 
            17:'septimum decimum', 18:'duodevicesimum', 19:'undevicesimum'
}



def get_mensis_from_month(month, year):
    if month == 1:
        mensis = menses['Ianuarius']
    if month == 2:
        if year % 400 == 0:
            mensis = menses['Februarius (leap year)']
        elif year % 4 == 0 and year % 100 != 0:
            mensis = menses['Februarius (leap year)']
        else: 
            mensis = menses['Februarius']
    if month == 3:
        mensis = menses['Martius']
    if month == 4:
        mensis = menses['Aprilis']
    if month == 5:
        mensis = menses['Maius']
    if month == 6:
        mensis = menses['Iunius']
    if month == 7:
        mensis = menses ['Iulius']
    if month == 8:
        mensis = menses['Augustus']
    if month == 9:
        mensis = menses['September']
    if month == 10:
        mensis = menses['October']
    if month == 11:
        mensis = menses['November']
    if month == 12:
        mensis = menses['December']
    return mensis

def get_mensis_prox(month, year):
    next_month = month+1
    mensis_prox = get_mensis_from_month(next_month, year)
    return mensis_prox

def get_datus(day, mensis, mensis_prox):
    if day == mensis['kalendae']:
        datus = f"\nKal. {mensis['abbr.']}\n(Kalendis {mensis['abl.pl.']})"
    elif day == mensis['nonae']:
        datus = f"\nNon. {mensis['abbr.']}\n(Nonis {mensis['abl.pl.']})"
    elif day == mensis['idus']:
        datus = f"\nEid. {mensis['abbr.']}\n(Idibus {mensis['abl.pl.']})"
    elif day == mensis['nonae']-1:
        datus = f"\nprid. Non. {mensis['abbr.']}\n(pridie Nonas {mensis['acc.fem.pl.']})"
    elif day == mensis['idus']-1:
        datus = f"\nprid. Non. {mensis['abbr.']}\n(pridie Idus {mensis['acc.fem.pl.']})"
    elif day == mensis['days']:
        datus = f"\nprid. Kal. {mensis_prox['abbr.']}\n(pridie Kalendas {mensis_prox['acc.fem.pl.']})"
    elif mensis == menses['Februarius (leap year)']:
        if day == 25:
            datus = "\na.d. VI Kal. Mar.\n(ante diem bis sextum Kalendas Martias)"
        elif mensis['idus'] < day < 25:
            number = mensis['days']-day+1
            numeral = num2roman(number)
            ordinal = ordinals[number]
            datus = f"\na.d. {numeral} Kal. Mar.\n(ante diem {ordinal} Kalendas Martias)"
        elif day > 25:
            number = mensis['days']-day+2
            numeral = num2roman(number)
            ordinal = ordinals[number]
            datus = f"\na.d. {numeral} Kal. Mar.\n(ante diem {ordinal} Kalendas Martias)"
    elif mensis['kalendae'] < day < mensis['nonae']-1:
        number = mensis['nonae']-day+1
        numeral = num2roman(number)
        ordinal = ordinals[number]
        datus = f"\na.d. {numeral} Non. {mensis['abbr.']}\n(ante diem {ordinal} Nonas {mensis['acc.fem.pl.']})"
    elif mensis['nonae'] < day < mensis['idus']:
        number = mensis['idus']-day+1
        numeral = num2roman(number)
        ordinal = ordinals[number]
        datus = f"\na.d. {numeral} Eid. {mensis['abbr.']}\n(ante diem {ordinal} Idus {mensis['acc.fem.pl.']})"
    elif day > mensis['idus']:
        number = mensis['days']-day+2
        numeral = num2roman(number)
        ordinal = ordinals[number]
        datus = f"\na.d. {numeral} Kal. {mensis_prox['abbr.']}\n(ante diem {ordinal} Kalendas {mensis_prox['acc.fem.pl.']})"

    return datus

mensis = get_mensis_from_month(month, year)
mensis_prox = get_mensis_prox(month, year)
datus = get_datus(day, mensis, mensis_prox)
print(datus)

## The Trouble with Timezones
Now it's time to deal with a challenge: the problem of dealing with different timezones. For most of my coding process, I thought I wouldn't have to deal with this issue. After all, I was receiving the sunrise and sunset times in UTC and finding our "timenow" term in UTC as well, cancelling each other out. Sure, "thedate" might be comically wrong in other timezones, but our modify_date() function took care of that as well. There seemed to be no issues with this. 

Then I noticed something strange when looking at locations near the International Date Line. Let's take Sydney, Australia as an example: 

In [ ]:
lat = '-33.8688'
lng = '151.2093'

timenowtz = datetime.datetime.now(timezone.utc)
timenow = timenowtz.replace(tzinfo=None)

thedate = timenow.date().strftime('%Y-%m-%d')

# Powered by SunriseSunset.io: https://sunrisesunset.io/api/
def make_sun_request(lat, lng):
    url = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={thedate}&time_format=24"
    r = requests.get(url)
    response = r.json()
    sunrise = response['results']['sunrise']
    sunset = response['results']['sunset']

    return [sunrise, sunset]

sunrise = make_sun_request(lat, lng)[0]
sunset = make_sun_request(lat, lng)[1]
print([sunrise, sunset])


Sunrise in Sydney, Australia is 8:30 PM in UTC, while sunset is 7:15 AM. This is to be expected, but leads to issues when we convert to datetimes: 

In [ ]:
risetime = datetime.datetime.strptime(f"{thedate}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
settime = datetime.datetime.strptime(f"{thedate}, {sunset}", "%Y-%m-%d, %H:%M:%S")
print(risetime)
print(settime)

They **don't** both occur on the same date UTC. And when we apply the same date to both of them, we end up with a sunset time that is before our sunrise time, which leads to all sorts of issues with our code. There's no simple way to fix this with the API. Although we can add the "&timezone=UTC" term to the url to get our times in UTC, the API still searches for the sunrise and sunset on the date inputed **as experienced locally.**

The first way I tried fixing this was by abandoning UTC and just using local time instead. This required installing some new Python modules: 

In [ ]:
%pip install pytz

In [ ]:
%pip install timezonefinder

In [ ]:
import pytz
from timezonefinder import TimezoneFinder

# Find local timezone
# Modified solution from here: https://stackoverflow.com/questions/15742045/getting-time-zone-from-lat-long-coordinates
# and from here: https://www.geeksforgeeks.org/get-current-time-in-different-timezone-using-python/
tf = TimezoneFinder()
    
latfl = float(lat)
lngfl =float(lng)

tz = tf.timezone_at(lng=lngfl, lat=latfl)
loc = pytz.timezone(tz)

# Assign times/dates for the rest of the operations
timenowtz = datetime.datetime.now(loc)
timenow = timenowtz.replace(tzinfo=None)
thedate = timenow.date().strftime('%Y-%m-%d')

This did actually function as expected, and seemingly works for every timezone, but brings with it issues of its own. Local timezones can be finicky to work with. Most notably, many countries have daylight savings of some kind, and on days when they are jumping an hour ahead or behind, the timedelta arithmetic will not account for that. UTC, being forever constant, is much preferred - if it can be made to work. 

As it turns out, it can! We can recognize when a change is being made by altering which dates (thedate, datebefore, dateafter) get added to which time when converting from strings to datetimes. If the sunrise > sunset (in 24-hour time) AND the longitude > zero (i.e. Eastern Hemisphere), we can shift our dates 1 forward, such that sunrise is attached to thedate and sunset is attached to dateafter. If the sunrise > sunset AND the longitude < zero (i.e. Western Hemisphere), shift our dates 1 backwards instead. 

A side effect of this is we actually need to broaden our range of times that we're getting. At 1:00 in the morning on March 15th UTC, it is still daytime on March 14th in Honolulu, while at 11:00 PM on March 15th UTC, it is daytime on March 16th in Australia. As such, we need to add in the sunrise of the day before and the sunset of the day after - which are 2 days before and 2 days after in UTC, respectively. 

When we make all these changes, we get something which looks like this: 

In [27]:
timenowtz = datetime.datetime.now(timezone.utc)
timenow = timenowtz.replace(tzinfo=None)

thedate = timenow.date().strftime('%Y-%m-%d')

daybefore = timenow - datetime.timedelta(days=1)
datebefore = daybefore.date().strftime('%Y-%m-%d')
daybefore2 = timenow - datetime.timedelta(days=2)
datebefore2 = daybefore.date().strftime('%Y-%m-%d')

dayafter = timenow + datetime.timedelta(days=1)
dateafter = dayafter.date().strftime('%Y-%m-%d')
dayafter2 = timenow + datetime.timedelta(days=2)
dateafter2 = dayafter.date().strftime('%Y-%m-%d')

# Powered by SunriseSunset.io: https://sunrisesunset.io/api/
def make_sun_request(lat, lng):
    url = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={thedate}&time_format=24"
    r = requests.get(url)
    response = r.json()
    sunrise = response['results']['sunrise']
    sunset = response['results']['sunset']

    url2 = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={datebefore}&time_format=24"
    r2 = requests.get(url2)
    response2 = r2.json()
    daystart = response2['results']['sunrise']
    nightstart = response2['results']['sunset']

    url3 = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={dateafter}&time_format=24"
    r3 = requests.get(url3)
    response3 = r3.json()
    nightend = response3['results']['sunrise']
    dayend = response3['results']['sunset']

    return [daystart, nightstart, sunrise, sunset, nightend, dayend]

def convert_sun_to_datetime(daystart, nightstart, sunrise, sunset, nightend, dayend, lng):
    splitrise = sunrise.split(':')
    hrise = splitrise[0]
    hrisefl = float(hrise)
    splitset = sunset.split(':')
    hset = splitset[0]
    hsetfl = float(hset)
    lngfl = float(lng)

    if hrisefl < hsetfl:
        firsttime = datetime.datetime.strptime(f"{datebefore}, {daystart}", "%Y-%m-%d, %H:%M:%S")
        secondtime = datetime.datetime.strptime(f"{datebefore}, {nightstart}", "%Y-%m-%d, %H:%M:%S")
        risetime = datetime.datetime.strptime(f"{thedate}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
        settime = datetime.datetime.strptime(f"{thedate}, {sunset}", "%Y-%m-%d, %H:%M:%S")
        penulttime = datetime.datetime.strptime(f"{dateafter}, {nightend}", "%Y-%m-%d, %H:%M:%S")
        lasttime = datetime.datetime.strptime(f"{dateafter}, {dayend}", "%Y-%m-%d, %H:%M:%S")
    
    if hrisefl > hsetfl:
        if lngfl>0:
            firsttime = datetime.datetime.strptime(f"{datebefore2}, {daystart}", "%Y-%m-%d, %H:%M:%S")
            secondtime = datetime.datetime.strptime(f"{datebefore}, {nightstart}", "%Y-%m-%d, %H:%M:%S")
            risetime = datetime.datetime.strptime(f"{datebefore}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
            settime = datetime.datetime.strptime(f"{thedate}, {sunset}", "%Y-%m-%d, %H:%M:%S")
            penulttime = datetime.datetime.strptime(f"{thedate}, {nightend}", "%Y-%m-%d, %H:%M:%S")
            lasttime = datetime.datetime.strptime(f"{dateafter}, {dayend}", "%Y-%m-%d, %H:%M:%S")
        if lngfl<0:
            firsttime = datetime.datetime.strptime(f"{datebefore}, {daystart}", "%Y-%m-%d, %H:%M:%S")
            secondtime = datetime.datetime.strptime(f"{thedate}, {nightstart}", "%Y-%m-%d, %H:%M:%S")
            risetime = datetime.datetime.strptime(f"{thedate}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
            settime = datetime.datetime.strptime(f"{dateafter}, {sunset}", "%Y-%m-%d, %H:%M:%S")
            penulttime = datetime.datetime.strptime(f"{dateafter}, {nightend}", "%Y-%m-%d, %H:%M:%S")
            lasttime = datetime.datetime.strptime(f"{dateafter2}, {dayend}", "%Y-%m-%d, %H:%M:%S")
    return [firsttime, secondtime, risetime, settime, penulttime, lasttime]

def get_datetimes_from_coord(lat, lng):
    srss = make_sun_request(lat, lng)
    daystart = srss[0]
    nightstart = srss[1]
    sunrise = srss[2]
    sunset = srss[3]
    nightend = srss[4]
    dayend = srss[5]
    datetimes = convert_sun_to_datetime(daystart, nightstart, sunrise, sunset, nightend, dayend, lng)
    return datetimes

A lot more intensive than before, but it works! The only thing this doesn't account for is that the International Date Line doesn't fall exactly on the 180/-180 longitudinal line. I hope the good people of Tonga, Samoa, Kiribati, and the Aleutian Islands can forgive me for this oversight.

## Finishing Touches and Putting It All Together
Now we can assemble our code! We'll add just a few more things to it as well. First, we'll add a function that can find the latitude and longitude given an ID number from the Digital Atlas of the Roman Empire. Then we'll add a function which prints our result nicely centered in an ASCII box, to make it more visually pleasing. Finally, we'll package the whole thing in prompts for user input, so that users can select their setting, refresh their clock, return to the menu to select a new setting, and quite the program.

The final result is below. Some suggestions for inputs can be found below it. 

In [ ]:
import datetime
from datetime import timezone
import requests

timenowtz = datetime.datetime.now(timezone.utc)
timenow = timenowtz.replace(tzinfo=None)

thedate = timenow.date().strftime('%Y-%m-%d')

daybefore = timenow - datetime.timedelta(days=1)
datebefore = daybefore.date().strftime('%Y-%m-%d')
daybefore2 = timenow - datetime.timedelta(days=2)
datebefore2 = daybefore.date().strftime('%Y-%m-%d')

dayafter = timenow + datetime.timedelta(days=1)
dateafter = dayafter.date().strftime('%Y-%m-%d')
dayafter2 = timenow + datetime.timedelta(days=2)
dateafter2 = dayafter.date().strftime('%Y-%m-%d')



# Powered by the Digital Atlas of the Roman Empire: https://imperium.ahlfeldt.se/
def make_coord_request(place_id):
    url = f"http://imperium.ahlfeldt.se/api/geojson.php?id={place_id}"
    r = requests.get(url)

    response = r.json()
    lat = response['features'][0]['geometry']['coordinates'][1]
    lng = response['features'][0]['geometry']['coordinates'][0]
    return [lat, lng]

# Powered by SunriseSunset.io: https://sunrisesunset.io/api/
def make_sun_request(lat, lng):
    url = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={thedate}&time_format=24"
    r = requests.get(url)
    response = r.json()
    sunrise = response['results']['sunrise']
    sunset = response['results']['sunset']

    url2 = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={datebefore}&time_format=24"
    r2 = requests.get(url2)
    response2 = r2.json()
    daystart = response2['results']['sunrise']
    nightstart = response2['results']['sunset']

    url3 = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={dateafter}&time_format=24"
    r3 = requests.get(url3)
    response3 = r3.json()
    nightend = response3['results']['sunrise']
    dayend = response3['results']['sunset']

    return [daystart, nightstart, sunrise, sunset, nightend, dayend]

def convert_sun_to_datetime(daystart, nightstart, sunrise, sunset, nightend, dayend, lng):
    splitrise = sunrise.split(':')
    hrise = splitrise[0]
    hrisefl = float(hrise)
    splitset = sunset.split(':')
    hset = splitset[0]
    hsetfl = float(hset)
    lngfl = float(lng)

    if hrisefl < hsetfl:
        firsttime = datetime.datetime.strptime(f"{datebefore}, {daystart}", "%Y-%m-%d, %H:%M:%S")
        secondtime = datetime.datetime.strptime(f"{datebefore}, {nightstart}", "%Y-%m-%d, %H:%M:%S")
        risetime = datetime.datetime.strptime(f"{thedate}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
        settime = datetime.datetime.strptime(f"{thedate}, {sunset}", "%Y-%m-%d, %H:%M:%S")
        penulttime = datetime.datetime.strptime(f"{dateafter}, {nightend}", "%Y-%m-%d, %H:%M:%S")
        lasttime = datetime.datetime.strptime(f"{dateafter}, {dayend}", "%Y-%m-%d, %H:%M:%S")
    
    if hrisefl > hsetfl:
        if lngfl>0:
            firsttime = datetime.datetime.strptime(f"{datebefore2}, {daystart}", "%Y-%m-%d, %H:%M:%S")
            secondtime = datetime.datetime.strptime(f"{datebefore}, {nightstart}", "%Y-%m-%d, %H:%M:%S")
            risetime = datetime.datetime.strptime(f"{datebefore}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
            settime = datetime.datetime.strptime(f"{thedate}, {sunset}", "%Y-%m-%d, %H:%M:%S")
            penulttime = datetime.datetime.strptime(f"{thedate}, {nightend}", "%Y-%m-%d, %H:%M:%S")
            lasttime = datetime.datetime.strptime(f"{dateafter}, {dayend}", "%Y-%m-%d, %H:%M:%S")
        if lngfl<0:
            firsttime = datetime.datetime.strptime(f"{datebefore}, {daystart}", "%Y-%m-%d, %H:%M:%S")
            secondtime = datetime.datetime.strptime(f"{thedate}, {nightstart}", "%Y-%m-%d, %H:%M:%S")
            risetime = datetime.datetime.strptime(f"{thedate}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
            settime = datetime.datetime.strptime(f"{dateafter}, {sunset}", "%Y-%m-%d, %H:%M:%S")
            penulttime = datetime.datetime.strptime(f"{dateafter}, {nightend}", "%Y-%m-%d, %H:%M:%S")
            lasttime = datetime.datetime.strptime(f"{dateafter2}, {dayend}", "%Y-%m-%d, %H:%M:%S")
    return [firsttime, secondtime, risetime, settime, penulttime, lasttime]

def get_datetimes_from_coord(lat, lng):
    srss = make_sun_request(lat, lng)
    daystart = srss[0]
    nightstart = srss[1]
    sunrise = srss[2]
    sunset = srss[3]
    nightend = srss[4]
    dayend = srss[5]
    datetimes = convert_sun_to_datetime(daystart, nightstart, sunrise, sunset, nightend, dayend, lng)
    return datetimes

def calc_hodie(risetime, settime):
    dies = settime-risetime
    hora = dies/12
    hora_prima = risetime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_secunda:
        tempus = "\nI\nprima diei hora"
    elif hora_secunda <= timenow < hora_tertia:
        tempus = "\nII\nseconda diei hora"
    elif hora_tertia <= timenow < hora_quarta:
        tempus = "\nIII\ntertia diei hora"
    elif hora_quarta <= timenow < hora_quinta:
        tempus = "\nIV\nquarta diei hora"
    elif hora_quinta <= timenow < hora_sexta:
        tempus = "\nV\nquinta diei hora"
    elif hora_sexta <= timenow < hora_septima:
        tempus = "\nVI\nsexta diei hora"
    elif hora_septima <= timenow < hora_octava:
        tempus = "\nVII\nseptima diei hora"
    elif hora_octava <= timenow < hora_nona:
        tempus = "\nVIII\noctava diei hora"
    elif hora_nona <= timenow < hora_decima:
        tempus = "\nIX\nnona diei hora"
    elif hora_decima <= timenow < hora_undecima:
        tempus = "\nX\ndecima diei hora"
    elif hora_undecima <= timenow < hora_duodecima:
        tempus = "\nXI\nundecima diei hora"
    elif hora_duodecima <= timenow < settime:
        tempus = "\nXII\nduodecima diei hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the day!"
    return tempus

def calc_heri(firsttime, secondtime):
    dies = secondtime-firsttime
    hora = dies/12
    hora_prima = firsttime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_secunda:
        tempus = "\nI\nprima diei hora"
    elif hora_secunda <= timenow < hora_tertia:
        tempus = "\nII\nseconda diei hora"
    elif hora_tertia <= timenow < hora_quarta:
        tempus = "\nIII\ntertia diei hora"
    elif hora_quarta <= timenow < hora_quinta:
        tempus = "\nIV\nquarta diei hora"
    elif hora_quinta <= timenow < hora_sexta:
        tempus = "\nV\nquinta diei hora"
    elif hora_sexta <= timenow < hora_septima:
        tempus = "\nVI\nsexta diei hora"
    elif hora_septima <= timenow < hora_octava:
        tempus = "\nVII\nseptima diei hora"
    elif hora_octava <= timenow < hora_nona:
        tempus = "\nVIII\noctava diei hora"
    elif hora_nona <= timenow < hora_decima:
        tempus = "\nIX\nnona diei hora"
    elif hora_decima <= timenow < hora_undecima:
        tempus = "\nX\ndecima diei hora"
    elif hora_undecima <= timenow < hora_duodecima:
        tempus = "\nXI\nundecima diei hora"
    elif hora_duodecima <= timenow < secondtime:
        tempus = "\nXII\nduodecima diei hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the day!"
    return tempus

def calc_cras(penulttime, lasttime):
    dies = lasttime-penulttime
    hora = dies/12
    hora_prima = penulttime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_secunda:
        tempus = "\nI\nprima diei hora"
    elif hora_secunda <= timenow < hora_tertia:
        tempus = "\nII\nseconda diei hora"
    elif hora_tertia <= timenow < hora_quarta:
        tempus = "\nIII\ntertia diei hora"
    elif hora_quarta <= timenow < hora_quinta:
        tempus = "\nIV\nquarta diei hora"
    elif hora_quinta <= timenow < hora_sexta:
        tempus = "\nV\nquinta diei hora"
    elif hora_sexta <= timenow < hora_septima:
        tempus = "\nVI\nsexta diei hora"
    elif hora_septima <= timenow < hora_octava:
        tempus = "\nVII\nseptima diei hora"
    elif hora_octava <= timenow < hora_nona:
        tempus = "\nVIII\noctava diei hora"
    elif hora_nona <= timenow < hora_decima:
        tempus = "\nIX\nnona diei hora"
    elif hora_decima <= timenow < hora_undecima:
        tempus = "\nX\ndecima diei hora"
    elif hora_undecima <= timenow < hora_duodecima:
        tempus = "\nXI\nundecima diei hora"
    elif hora_duodecima <= timenow < lasttime:
        tempus = "\nXII\nduodecima diei hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the day!"
    return tempus

# quid proxima, quid superiore nocte egeris (Cic. Cat. 1.1.1)
def calc_nox_proxima(startime, risetime):
    nox = risetime - startime
    hora = nox/12
    hora_prima = startime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_quarta:
        watch = "\nVIGILIA PRIMA\n"
        if hora_prima <= timenow < hora_secunda:
            tempus = watch + "I\nprima noctis hora"
        elif hora_secunda <= timenow < hora_tertia:
            tempus = watch + "II\nseconda noctis hora"
        elif hora_tertia <= timenow < hora_quarta:
            tempus = watch + "III\ntertia noctis hora"
    elif hora_quarta <= timenow < hora_septima:
        watch = "\nVIGILIA SECUNDA\n"
        if hora_quarta <= timenow < hora_quinta:
            tempus = watch + "IV\nquarta noctis hora"
        elif hora_quinta <= timenow < hora_sexta:
            tempus = watch + "V\nquinta noctis hora"
        elif hora_sexta <= timenow < hora_septima:
            tempus = watch + "VI\nsexta noctis hora"
    elif hora_septima <= timenow < hora_decima:
        watch = "\nVIGILIA TERTIA\n"
        if hora_septima <= timenow < hora_octava:
            tempus = watch + "VII\nseptima noctis hora"
        elif hora_octava <= timenow < hora_nona:
            tempus = watch + "VIII\noctava noctis hora"
        elif hora_nona <= timenow < hora_decima:
            tempus = watch + "IX\nnona noctis hora"
    elif hora_decima <= timenow < risetime:
        watch = "\nVIGILIA QUARTA\n"
        if hora_decima <= timenow < hora_undecima:
            tempus = watch + "X\ndecima noctis hora"
        elif hora_undecima <= timenow < hora_duodecima:
            tempus = watch + "XI\nundecima noctis hora"
        elif hora_duodecima <= timenow < risetime:
            tempus = watch + "XII\nduodecima noctis hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the last night!"
    return tempus

def calc_nox_superior(settime, penulttime):
    nox = penulttime - settime
    hora = nox/12
    hora_prima = settime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_quarta:
        watch = "\n  VIGILIA PRIMA\n"
        if hora_prima <= timenow < hora_secunda:
            tempus = watch + "        I\nprima noctis hora"
        elif hora_secunda <= timenow < hora_tertia:
            tempus = watch + "II\nseconda noctis hora"
        elif hora_tertia <= timenow < hora_quarta:
            tempus = watch + "III\ntertia noctis hora"
    elif hora_quarta <= timenow < hora_septima:
        watch = "\nVIGILIA SECUNDA\n"
        if hora_quarta <= timenow < hora_quinta:
            tempus = watch + "IV\nquarta noctis hora"
        elif hora_quinta <= timenow < hora_sexta:
            tempus = watch + "V\nquinta noctis hora"
        elif hora_sexta <= timenow < hora_septima:
            tempus = watch + "VI\nsexta noctis hora"
    elif hora_septima <= timenow < hora_decima:
        watch = "\nVIGILIA TERTIA\n"
        if hora_septima <= timenow < hora_octava:
            tempus = watch + "VII\nseptima noctis hora"
        elif hora_octava <= timenow < hora_nona:
            tempus = watch + "VIII\noctava noctis hora"
        elif hora_nona <= timenow < hora_decima:
            tempus = watch + "IX\nnona noctis hora"
    elif hora_decima <= timenow < settime:
        watch = "\nVIGILIA QUARTA\n"
        if hora_decima <= timenow < hora_undecima:
            tempus = watch + "X\ndecima noctis hora"
        elif hora_undecima <= timenow < hora_duodecima:
            tempus = watch + "XI\nundecima noctis hora"
        elif hora_duodecima <= timenow < settime:
            tempus = watch + "XII\nduodecima noctis hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the next night!"
    return tempus 

def get_tempus_from_coord(lat, lng):
    datetimes = get_datetimes_from_coord(lat, lng)
    firsttime = datetimes[0]
    secondtime = datetimes[1]
    risetime = datetimes[2]
    settime = datetimes[3]
    penulttime = datetimes[4]
    lasttime = datetimes[5]
    if firsttime <= timenow < secondtime:
        heri = calc_heri(firsttime, secondtime)
        return heri
    elif secondtime <= timenow < risetime:
        nox_proxima = calc_nox_proxima(secondtime, risetime)
        return nox_proxima
    elif risetime <= timenow < settime:
        hodie = calc_hodie(risetime, settime)
        return hodie
    elif settime <= timenow < penulttime:
        nox_superior = calc_nox_superior(settime, penulttime)
        return nox_superior
    elif penulttime <= timenow < lasttime:
        cras = calc_cras(penulttime, lasttime)
        return cras
    else:
        return "Eheu! Couldn't get the time from the sun"



def modify_date(firsttime, secondtime, risetime, settime, penulttime, lasttime):
    if secondtime <= timenow < risetime:
        nox_proxima = risetime - secondtime
        dimidium = nox_proxima/2
        media_nox = secondtime + dimidium
        if secondtime <= timenow < media_nox:
            mod_date = datebefore
        else:
            mod_date = thedate
    elif settime <= timenow < penulttime:
        nox_superior = penulttime - settime
        dimidium = nox_superior/2
        media_nox = settime + dimidium
        if media_nox <= timenow < penulttime:
            mod_date = dateafter
        else:
            mod_date = thedate
    elif risetime <= timenow < settime:
        mod_date = thedate
    elif firsttime <= timenow < secondtime:
        mod_date = datebefore
    elif penulttime <= timenow < lasttime:
        mod_date = dateafter
    else:
        mod_date = "Eheu! Couldn't figure out if it's past midnight!"
    return mod_date


def split_date(mod_date):
    splitdate = mod_date.split("-")
    yearstr = splitdate[0]
    year = int(yearstr)
    monthstr = splitdate[1]
    month = int(monthstr)
    daystr = splitdate[2]
    day = int(daystr)
    return [year, month, day]

menses = {
    'Ianuarius': {
        'name': 'Ianuarius',
        'abbr.':'Ian.',
        'nom.fem.pl.': 'Ianuariae',
        'acc.fem.pl.': 'Ianuarias',
        'abl.pl.': 'Ianuariis',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Februarius': {
        'name': 'Februarius', 
        'abbr.':'Feb.',
        'nom.fem.pl.': 'Februariae',
        'acc.fem.pl.': 'Februarias',
        'abl.pl.': 'Februariis',
        'days': 28, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Februarius (leap year)': {
        'name': 'Februarius', 
        'abbr.':'Feb.',
        'nom.fem.pl.': 'Februariae',
        'acc.fem.pl.': 'Februarias',
        'abl. pl.': 'Februariis',
        'days': 29, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Martius': {
        'name': 'Martius', 
        'abbr.':'Mar.',
        'nom.fem.pl.': 'Martiae',
        'acc.fem.pl.': 'Martias',
        'abl.pl.': 'Martiis',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 7, 
        'idus': 15
        },
    'Aprilis': {
        'name': 'Aprilis', 
        'abbr.':'Apr.',
        'nom.fem.pl.': 'Apriles',
        'acc.fem.pl.': 'Apriles',
        'abl.pl.': 'Aprilibus',
        'days': 30, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Maius': {
        'name': 'Maius', 
        'abbr.':'Mai.',
        'nom.fem.pl.': 'Maiae',
        'acc.fem.pl.': 'Maias',
        'abl.pl.': 'Maiis',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 7, 
        'idus': 15
        },
    'Iunius': {
        'name': 'Iunius',
        'abbr.':'Iun.',
        'nom.fem.pl.': 'Iuniae',
        'acc.fem.pl.': 'Iunias', 
        'abl.pl.': 'Iuniis',
        'days': 30, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'Iulius': {
        'name': 'Iulius', 
        'abbr.':'Iul.',
        'nom.fem.pl.': 'Iuliae',
        'acc.fem.pl.': 'Iulias',
        'abl.pl.': 'Iuliis',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 7, 
        'idus': 15
        },
    'Augustus': {
        'name': 'Augustus',
        'abbr.':'Aug.',
        'nom.fem.pl.': 'Augustae',
        'acc.fem.pl.': 'Augustas',
        'abl.pl.': 'Augustis', 
        'days': 31, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'September': {
        'name': 'September',
        'abbr.':'Sept.',
        'nom.fem.pl.': 'Septembres',
        'acc.fem.pl.': 'Septembres',
        'abl.pl.': 'Septembribus', 
        'days': 30, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'October': {
        'name': 'October',
        'abbr.':'Oct.',
        'nom.fem.pl.': 'Octobres',
        'acc.fem.pl.': 'Octobres',
        'abl.pl.': 'Octobribus', 
        'days': 31, 
        'kalendae': 1, 
        'nonae': 7, 
        'idus': 15
        },
    'November': {
        'name': 'November',
        'abbr.':'Nov.',
        'nom.fem.pl.': 'Novembres',
        'acc.fem.pl.': 'Novembres', 
        'abl.pl.': 'Novembribus',
        'days': 30, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13
        },
    'December': {
        'name': 'December', 
        'abbr.':'Dec.',
        'nom.fem.pl.': 'Decembres',
        'acc.fem.pl.': 'Decembres',
        'abl.pl.': 'Decembribus',
        'days': 31, 
        'kalendae': 1, 
        'nonae': 5, 
        'idus': 13}
        }

# Solution found here: https://stackoverflow.com/questions/28777219/basic-program-to-convert-integer-to-roman-numerals
num_map = [(1000, 'M'), (900, 'CM'), (500, 'D'), (400, 'CD'), (100, 'C'), (90, 'XC'),
           (50, 'L'), (40, 'XL'), (10, 'X'), (9, 'IX'), (5, 'V'), (4, 'IV'), (1, 'I')]

def num2roman(num):
    roman = ''
    while num > 0:
        for i, r in num_map:
            while num >= i:
                roman += r
                num -= i

    return roman


ordinals = {1:'primum', 2:'secundum', 3:'tertium', 4:'quartum', 5:'quintum', 6:'sextum', 
            7:'septimum', 8:'octavum', 9:'nonum', 10:'decimum', 11:'undecimum', 12:'duodecimum', 
            13:'tertium decimum', 14:'quartum decimum', 15:'quintum decimum', 16:'sextum decimum', 
            17:'septimum decimum', 18:'duodevicesimum', 19:'undevicesimum'
}



def get_mensis_from_month(month, year):
    if month == 1:
        mensis = menses['Ianuarius']
    if month == 2:
        if year % 400 == 0:
            mensis = menses['Februarius (leap year)']
        elif year % 4 == 0 and year % 100 != 0:
            mensis = menses['Februarius (leap year)']
        else: 
            mensis = menses['Februarius']
    if month == 3:
        mensis = menses['Martius']
    if month == 4:
        mensis = menses['Aprilis']
    if month == 5:
        mensis = menses['Maius']
    if month == 6:
        mensis = menses['Iunius']
    if month == 7:
        mensis = menses ['Iulius']
    if month == 8:
        mensis = menses['Augustus']
    if month == 9:
        mensis = menses['September']
    if month == 10:
        mensis = menses['October']
    if month == 11:
        mensis = menses['November']
    if month == 12:
        mensis = menses['December']
    return mensis

def get_mensis_prox(month, year):
    next_month = month+1
    mensis_prox = get_mensis_from_month(next_month, year)
    return mensis_prox

def get_datus(day, mensis, mensis_prox):
    if day == mensis['kalendae']:
        datus = f"\nKal. {mensis['abbr.']}\n(Kalendis {mensis['abl.pl.']})"
    elif day == mensis['nonae']:
        datus = f"\nNon. {mensis['abbr.']}\n(Nonis {mensis['abl.pl.']})"
    elif day == mensis['idus']:
        datus = f"\nEid. {mensis['abbr.']}\n(Idibus {mensis['abl.pl.']})"
    elif day == mensis['nonae']-1:
        datus = f"\nprid. Non. {mensis['abbr.']}\n(pridie Nonas {mensis['acc.fem.pl.']})"
    elif day == mensis['idus']-1:
        datus = f"\nprid. Non. {mensis['abbr.']}\n(pridie Idus {mensis['acc.fem.pl.']})"
    elif day == mensis['days']:
        datus = f"\nprid. Kal. {mensis_prox['abbr.']}\n(pridie Kalendas {mensis_prox['acc.fem.pl.']})"
    elif mensis == menses['Februarius (leap year)']:
        if day == 25:
            datus = "\na.d. VI Kal. Mar.\n(ante diem bis sextum Kalendas Martias)"
        elif mensis['idus'] < day < 25:
            number = mensis['days']-day+1
            numeral = num2roman(number)
            ordinal = ordinals[number]
            datus = f"\na.d. {numeral} Kal. Mar.\n(ante diem {ordinal} Kalendas Martias)"
        elif day > 25:
            number = mensis['days']-day+2
            numeral = num2roman(number)
            ordinal = ordinals[number]
            datus = f"\na.d. {numeral} Kal. Mar.\n(ante diem {ordinal} Kalendas Martias)"
    elif mensis['kalendae'] < day < mensis['nonae']-1:
        number = mensis['nonae']-day+1
        numeral = num2roman(number)
        ordinal = ordinals[number]
        datus = f"\na.d. {numeral} Non. {mensis['abbr.']}\n(ante diem {ordinal} Nonas {mensis['acc.fem.pl.']})"
    elif mensis['nonae'] < day < mensis['idus']:
        number = mensis['idus']-day+1
        numeral = num2roman(number)
        ordinal = ordinals[number]
        datus = f"\na.d. {numeral} Eid. {mensis['abbr.']}\n(ante diem {ordinal} Idus {mensis['acc.fem.pl.']})"
    elif day > mensis['idus']:
        number = mensis['days']-day+2
        numeral = num2roman(number)
        ordinal = ordinals[number]
        datus = f"\na.d. {numeral} Kal. {mensis_prox['abbr.']}\n(ante diem {ordinal} Kalendas {mensis_prox['acc.fem.pl.']})"

    return datus



def horologium_universalis(lat, lng):
    tempus = get_tempus_from_coord(lat, lng)
    datetimes = get_datetimes_from_coord(lat, lng)
    firsttime = datetimes[0]
    secondtime = datetimes[1]
    risetime = datetimes[2]
    settime = datetimes[3]
    penulttime = datetimes[4]
    lasttime = datetimes[5]
    mod_date = modify_date(firsttime, secondtime, risetime, settime, penulttime, lasttime)
    ymd = split_date(mod_date)
    year = ymd[0]
    month = ymd[1]
    day = ymd[2]
    mensis = get_mensis_from_month(month, year)
    mensis_prox = get_mensis_prox(month, year)
    datus = get_datus(day, mensis, mensis_prox)
    datus_et_tempus = f"{datus}\n{tempus}\n"
    return datus_et_tempus


def horologium_romanum(place_id):
    coords = make_coord_request(place_id)
    lat = coords[0]
    lng = coords[1]
    datus_et_tempus = horologium_universalis(lat, lng)
    return datus_et_tempus

# Solution found here: https://www.reddit.com/r/learnpython/comments/1c19y94/learning_dynamic_text_box/
def print_in_box(text: str) -> None:
    """Print multi-line text in a box."""
    margin_width = 2
    horizontal_border_char = '='
    vertical_border_char = '|'

    lines = text.split('\n')
    max_line_length = max(len(line) for line in lines)

    max_line_length += 2 * margin_width
    horizontal_border = (
        vertical_border_char +
        horizontal_border_char * max_line_length +
        vertical_border_char
        )

    print(horizontal_border)

    for line in lines:
        # Calculate margin widths.
        left_margin = (max_line_length - len(line)) // 2
        right_margin = max_line_length - (len(line) + left_margin)

        formatted_line = (
            f"{vertical_border_char}"
            f"{' ' * left_margin}{line}{' ' * right_margin}"
            f"{vertical_border_char}"
            )

        print(formatted_line)

    print(horizontal_border)

def horologium():
    while True:
        setting = input("""
Salve! What setting would you like to use? To pick a location from the Digital Atlas of the Roman Empire (https://imperium.ahlfeldt.se/), enter 1. To use any latitude and longitude, enter 2. To quit, enter 0.
        """)

        if setting == '1':
            place_id = input("Please enter the place ID of a location from the Digital Atlas of the Roman Empire:")
            horologium = horologium_romanum(place_id)
            print("\n\n")
            print_in_box(horologium)

            while True: 
                refresh = input("\n\n\n To refresh, press Enter. To select a new location, enter 1.")
                if refresh == '1':
                    break
                else:
                    horologium = horologium_romanum(place_id)
                    print("\n\n")
                    print_in_box(horologium)

        elif setting == '2':
            lat = input("Please enter your latitude, in decimal notation:")
            lng = input("Please enter your longitude, in decimal notation:")
            horologium = horologium_universalis(lat, lng)
            print("\n\n")
            print_in_box(horologium)

            while True: 
                refresh = input("\n\n\n To refresh, press Enter. To select a new location, enter 1.")
                if refresh == '1':
                    break
                else:
                    horologium = horologium_universalis(lat, lng)
                    print("\n\n")
                    print_in_box(horologium)
        elif setting == '0':
            print('Vale!')
            break

horologium()

**For the DARE:**

Rome: 1438

Athens: 10975

Alexandria: 15898

**For custom latitude and longitude:**

Tufts University: 42.405547, -71.120259

Mexico City: 19.4326, -99.1332

Helsinki, Finland: 60.1699, 24.9384

Honolulu, Hawaii: 21.315603, -157.858093

## Final Thoughts

While one could create an analytical tool surrounding Roman timekeeping in the form of something like a converter, my objective here was to create something which can link users experientially in a very small way to antiquity. Personally, when checking this clock for my local location periodically, I've noticed just how skewed my own life is from the rhythms of life of the ancient world. While a Roman person would likely have experienced all twelve hours of their day, I tend to sleep through the first several hours of daylight and continue to be awake and work well into the hours of the night. Having a clock which is connected to the patterns of the sun, rather than simply abstract numbers, reminds us of how our pre-industrial ancestors tended to live their lives. 

More features could certainly be added to this. I wanted to package this as a command line application, but looking up how to do so, it seemed like a surprisingly complicated process, and I didn't end up having time to do it. If I gain better experience or find someone else interested in the project, however, I think it could work even better with a GUI. I also think there could be more user-friendly ways to select a location, such as by searching for a location by name or even using location services to use one's own location. Unfortunately, those are way beyond my skills. 

Building this digital *horologium* was an incredibly fun and educational experience, and I feel that it really helped me to hone my skills with Python. 

## Bibliography
Adam, Alexander. *Roman Antiquities, or an Account of the Manners and Customs of the Romans.* Edinburgh, 1791. https://books.google.com/books?id=-tdRAAAAcAAJ&printsec=frontcover&source=gbs_ge_summary_r&cad=0#v=onepage&q&f=false

Beck, Charles. *Syntax of the Latin Language, Chiefly from the German of C. G. Zumpt.* James Munroe and Company, 1844. https://books.google.com/books?id=uJgQAAAAYAAJ&pg=PA177#v=onepage&q&f=false

Censorinus. *De Die Natali Liber.* https://penelope.uchicago.edu/Thayer/L/Roman/Texts/Censorinus/text*.html. 

Censorinus. *De Die Natali.* Translated by William Maude. The Cambridge Encyclopedia Co., 1900. https://elfinspell.com/ClassicalTexts/Maude/Censorinus/DeDieNatale-Part2.html#topref107.

"The Roman Calendar." University of Chicago. https://penelope.uchicago.edu/encyclopaedia_romana/calendar/romancalendar.html. 

Schmitz, Leonhard. "Hora." In *A Dictionary of Greek and Roman Antiquities,* edited by William Smith. John Murray, 1875. https://penelope.uchicago.edu/Thayer/E/Roman/Texts/secondary/SMIGRA*/Hora.html.
